# Split data

In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import collections

# Load the dataset
data_dir = 'data'

# data = pd.read_csv(f'{data_dir}/audio_features_no_duplicates_new_normalized.csv')
data = pd.read_csv(f'{data_dir}/audio_features_no_duplicates_new.csv')

In [99]:
# Exclude the first column (filename) and assume the last column is the label
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

# Check the distribution of labels
label_distribution = collections.Counter(y)
print(f"Label distribution: {label_distribution}")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Label distribution: Counter({0: 1752, 1: 1678})


# Random Forrest Model

In [121]:
from sklearn.ensemble import RandomForestClassifier

# Train the classifier
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)

In [120]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 86.30%
Precision: 0.86
Recall: 0.86
F1 Score: 0.86
Confusion Matrix:
[[430  73]
 [ 68 458]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       503
           1       0.86      0.87      0.87       526

    accuracy                           0.86      1029
   macro avg       0.86      0.86      0.86      1029
weighted avg       0.86      0.86      0.86      1029



In [ ]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import librosa
import numpy as np

def extract_features(file_path):
    
    # Load the audio file
    y, sr = librosa.load(file_path)

    # Extract features
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    zero_crossings = np.mean(librosa.feature.zero_crossing_rate(y))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rms_energy = np.mean(librosa.feature.rms(y=y))

    # Combine all features into a single array
    features = np.hstack([mfccs, chroma, spectral_contrast, zero_crossings, spectral_bandwidth, rms_energy])

    return features

def predict_track(file_path, model):

    # Extract features from the audio file
    features = extract_features(file_path)
    
    # Convert features to a DataFrame with the same columns as used in training
    feature_df = pd.DataFrame([features])

    
    # Make a prediction and get probabilities
    label = model.predict(feature_df)
    probabilities = model.predict_proba(feature_df)
    
    return label,probabilities


## Getting predictions on unseen track

In [114]:
import os

track_directory = 'data/'

# Loop through all files in the directory
for track_name in os.listdir(track_directory):
    if track_name.endswith('.mp3'):
        track_path = os.path.join(track_directory, track_name)

        label, prediction = predict_track(track_path, clf)

        if label == 0:
            print(f"{track_name} is not proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")
        else:
            print(f"{track_name} is proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")

Rene Wise - Lakota Fox.mp3 is proper! Label is 1 with a probability of 0.885
Kike Pravda - Main Control [tZcM4ytxSGc].mp3 is proper! Label is 1 with a probability of 0.865
Beatrice - I'm Not From Here.mp3 is proper! Label is 1 with a probability of 0.76
Lowerzone - Verknipt People.mp3 is not proper! Label is 0 with a probability of 0.675
Mark Dekoda - Rave Harder Techno Bass.mp3 is not proper! Label is 0 with a probability of 0.62
09 - Step To Enchantment (Stringent).mp3 is proper! Label is 1 with a probability of 0.685
Kaiser (K S R) - Driving In A Fast Tool.mp3 is proper! Label is 1 with a probability of 0.905
Jeff Mills - Step to enchantment [eLtnhsLwSZE].mp3 is not proper! Label is 0 with a probability of 0.595
Voorman - Surface Scatter.mp3 is proper! Label is 1 with a probability of 0.87
Stef Mendesidis - Crime and Punishment.mp3 is proper! Label is 1 with a probability of 0.775
Reinier Zonneveld - Mom Was On Tequila.mp3 is not proper! Label is 0 with a probability of 0.97
Alexand

In [110]:
# Get prediction over one track
track_name = "09 - Step To Enchantment (Stringent).mp3"
track_path = os.path.join(track_directory, track_name)
label, prediction = predict_track(track_path, clf)

print(f"{track_name} is {label[0]} with a probability of {prediction[0][label[0]]}")

09 - Step To Enchantment (Stringent).mp3 is 1 with a probability of 0.73


## Notes

Looks like the model's predictions are only reliable when the quality of the track is not totally messed up. Needs to be decent quality. Not noisy vinyl rips.

When we deploy the model as a web app, and users can submit a Youtube link to a track, downloading the audio in high quality will be problematic. 

We need to find a way to download the audio in high quality. Or, we should downgrade the training data to match the quality of the audio that users can submit from Youtube.



## Saving the model

In [96]:
import pickle

# Save the model
with open('output/random_forest_model.pkl', 'wb') as file:
    pickle.dump(clf, file)

# # Load the model
# with open('random_forest_model.pkl', 'rb') as file:
#     clf_loaded = pickle.load(file)

# XGBoost Model

In [123]:
from xgboost import XGBClassifier

# Initialize and train the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred))

/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:15:47] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.8775510204081632
XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.87       503
           1       0.88      0.88      0.88       526

    accuracy                           0.88      1029
   macro avg       0.88      0.88      0.88      1029
weighted avg       0.88      0.88      0.88      1029



In [124]:
import os

track_directory = 'data/'

# Loop through all files in the directory
for track_name in os.listdir(track_directory):
    if track_name.endswith('.mp3'):
        track_path = os.path.join(track_directory, track_name)

        label, prediction = predict_track(track_path, xgb_model)

        if label == 0:
            print(f"{track_name} is not proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")
        else:
            print(f"{track_name} is proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")

Rene Wise - Lakota Fox.mp3 is proper! Label is 1 with a probability of 0.9866272807121277
Kike Pravda - Main Control [tZcM4ytxSGc].mp3 is proper! Label is 1 with a probability of 0.9976605176925659
Beatrice - I'm Not From Here.mp3 is proper! Label is 1 with a probability of 0.9985795021057129
Lowerzone - Verknipt People.mp3 is not proper! Label is 0 with a probability of 0.9979397654533386
Mark Dekoda - Rave Harder Techno Bass.mp3 is not proper! Label is 0 with a probability of 0.9347617626190186
09 - Step To Enchantment (Stringent).mp3 is proper! Label is 1 with a probability of 0.998744010925293
Kaiser (K S R) - Driving In A Fast Tool.mp3 is proper! Label is 1 with a probability of 0.997433602809906
Jeff Mills - Step to enchantment [eLtnhsLwSZE].mp3 is not proper! Label is 0 with a probability of 0.9096932411193848
Voorman - Surface Scatter.mp3 is proper! Label is 1 with a probability of 0.9994863271713257
Stef Mendesidis - Crime and Punishment.mp3 is proper! Label is 1 with a probab

# Logistic Regression, SVM, KNN

In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Train Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

y_pred_logistic = logistic_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_logistic)
precision = precision_score(y_test, y_pred_logistic, average='weighted')
recall = recall_score(y_test, y_pred_logistic, average='weighted')
f1 = f1_score(y_test, y_pred_logistic, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred_logistic)
class_report = classification_report(y_test, y_pred_logistic)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 84.74%
Precision: 0.85
Recall: 0.85
F1 Score: 0.85
Confusion Matrix:
[[428  75]
 [ 82 444]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       503
           1       0.86      0.84      0.85       526

    accuracy                           0.85      1029
   macro avg       0.85      0.85      0.85      1029
weighted avg       0.85      0.85      0.85      1029



/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [129]:
# Train SVM
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_svm)
precision = precision_score(y_test, y_pred_svm, average='weighted')
recall = recall_score(y_test, y_pred_svm, average='weighted')
f1 = f1_score(y_test, y_pred_svm, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred_svm)
class_report = classification_report(y_test, y_pred_svm)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 78.33%
Precision: 0.79
Recall: 0.78
F1 Score: 0.78
Confusion Matrix:
[[432  71]
 [152 374]]
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.86      0.79       503
           1       0.84      0.71      0.77       526

    accuracy                           0.78      1029
   macro avg       0.79      0.78      0.78      1029
weighted avg       0.79      0.78      0.78      1029



In [127]:
# Train KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

y_pred_knn = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_knn)
precision = precision_score(y_test, y_pred_knn, average='weighted')
recall = recall_score(y_test, y_pred_knn, average='weighted')
f1 = f1_score(y_test, y_pred_knn, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred_knn)
class_report = classification_report(y_test, y_pred_knn)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 81.73%
Precision: 0.82
Recall: 0.82
F1 Score: 0.82
Confusion Matrix:
[[424  79]
 [109 417]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       503
           1       0.84      0.79      0.82       526

    accuracy                           0.82      1029
   macro avg       0.82      0.82      0.82      1029
weighted avg       0.82      0.82      0.82      1029



/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [130]:
import os

track_directory = 'data/'

for model in [logistic_model, svm_model, knn_model]:
    print(f"Model: {model}")

    # Loop through all files in the directory
    for track_name in os.listdir(track_directory):
        if track_name.endswith('.mp3'):
            track_path = os.path.join(track_directory, track_name)

            label, prediction = predict_track(track_path, model)

            if label == 0:
                print(f"{track_name} is not proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")
            else:
                print(f"{track_name} is proper! Label is {label[0]} with a probability of {prediction[0][label[0]]}")

Model: LogisticRegression()
Rene Wise - Lakota Fox.mp3 is proper! Label is 1 with a probability of 0.8934672488502142
Kike Pravda - Main Control [tZcM4ytxSGc].mp3 is proper! Label is 1 with a probability of 0.8806780022623224
Beatrice - I'm Not From Here.mp3 is proper! Label is 1 with a probability of 0.9493046722020121
Lowerzone - Verknipt People.mp3 is not proper! Label is 0 with a probability of 0.5532544823335399
Mark Dekoda - Rave Harder Techno Bass.mp3 is not proper! Label is 0 with a probability of 0.7003384048900274
09 - Step To Enchantment (Stringent).mp3 is proper! Label is 1 with a probability of 0.8382499301230181
Kaiser (K S R) - Driving In A Fast Tool.mp3 is proper! Label is 1 with a probability of 0.888100460451353
Jeff Mills - Step to enchantment [eLtnhsLwSZE].mp3 is proper! Label is 1 with a probability of 0.5456870861280316
Voorman - Surface Scatter.mp3 is proper! Label is 1 with a probability of 0.9614647768970976
Stef Mendesidis - Crime and Punishment.mp3 is proper!

/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Rene Wise - Lakota Fox.mp3 is proper! Label is 1 with a probability of 1.0


/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Kike Pravda - Main Control [tZcM4ytxSGc].mp3 is proper! Label is 1 with a probability of 0.8


...

/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Reinier Zonneveld - Mom Was On Tequila.mp3 is not proper! Label is 0 with a probability of 0.8


/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Alexander Johansson   Mattias Fridell - Retsticka.mp3 is proper! Label is 1 with a probability of 0.6
Mike Parker - Rainmaker.mp3 is proper! Label is 1 with a probability of 0.8


/Users/zein/opt/miniconda3/envs/proper-classifier/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# Spectrogram based CNN Model

## Training the CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: proper and improper, so 2 output neurons
])

#Load images and prepare the training and testing data


# Preprocess your data
# Assuming X_train and X_test are your image data and y_train, y_test are your labels
# Ensure X_train and X_test are reshaped to (num_samples, 128, 128, 3) and normalized
X_train = X_train.reshape(-1, 128, 128, 3) / 255.0
X_test = X_test.reshape(-1, 128, 128, 3) / 255.0

# Convert labels to categorical if they are not already
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
